
<div>
    <h1 align=center style="font-size: 12; font-weight: 900; ">--- 10 ---<h1>
    <h2 align=center>CSiAPI - ETABS y SAP200</h2>
    <!-- <p><strong></strong</p> -->
    <p align="center"><font color = orange >Bach. Ing. Francisco Jimenez Mucho (fjmucho0@gmail.com)</font></p> 
    <p align="center"><font color = green >2022, update: 2025</font></p> 
</div>

---

## Trabajo con reporte de datos de ETABS o SAP2000, ...

Solicitar informacion a ETABS o SAP200 para filtrar y hacer reporte personalizado de `Datos`

### 1. Dependecnias

In [34]:
import os, sys

import numpy as np, pandas as pd

import ipywidgets as widgets

# from matplotlib import pyplot as plt

In [35]:
import comtypes.client

<!-- ### 2. Importar la funcion o metodo para conectarnos y poder solicitar informacion -->
### 2. Seleccion de la apliacion

In [36]:
# from CSiAPI.ConnectCSiAPI import metodo
# Indicar el programa a usar
connect_to = 2; #1 para SAP2000 y 2 para ETABS

connect_to_app, smodel = None, None

In [37]:
conn = {};
if connect_to == 1:  # SAP2000
    name_app = "SAP2000"
    conn = {
        'app_ruta': "C:\\Program Files\\Computers and Structures\\SAP2000 23\\SAP2000.exe",
        'app_adjunto': "CSI.SAP2000.API.SapObject",
        'app_helper': 'SAP2000v1.Helper'
        }
elif (connect_to == 2): # ETABS
    name_app = "ETABS"
    conn = {
        'app_ruta': "C:\\Program Files\\Computers and Structures\\ETABS 21\\ETABS.exe",
        'app_adjunto': "CSI.ETABS.API.ETABSObject",
        'app_helper': "ETABSv1.Helper"
        }
else: 
    print("No tenemos soporte aun")
    sys.exit()

# match connect_to:
#     case 1:
#         name_app = "SAP2000"
#         conn = {
#             'app_ruta': "C:\\Program Files\\Computers and Structures\\SAP2000 23\\SAP2000.exe",
#             'app_adjunto': "CSI.SAP2000.API.SapObject",
#             'app_helper': 'SAP2000v1.Helper'
#         }
#     case 2:
#         name_app = "ETABS"
#         conn = {
#             'app_ruta': "C:\\Program Files\\Computers and Structures\\ETABS 21\\ETABS.exe",
#             'app_adjunto': "CSI.ETABS.API.ETABSObject",
#             'app_helper': "ETABSv1.Helper"
#         }

### 3. Coneccion a la aplicacion

**Primera forma**

In [38]:
try:
    connect_to_app = comtypes.client.GetActiveObject(conn["app_adjunto"])
    print(f"Instancia establecida (adjuntada) a {name_app}")
except (OSError, comtypes.COMError):
    print("No running instance of the program found or failed to attach.")
    sys.exit(-1)

Instancia establecida (adjuntada) a ETABS


**Segunda forma**

In [39]:
# helper = comtypes.client.CreateObject(conn["app_helper"])
# if connect_to == 1: 
#     helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)
# else: 
#     helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper);
# try:
#     connect_to_app = helper.GetObject(conn["app_adjunto"])
#     print(f"Instancia establecida (adjuntada) a {name_app}")
# except (OSError, comtypes.COMError):
#     print("No running instance of the program found or failed to attach.")
#     sys.exit(-1)

### 4. Se crea el objeto SapModel o instancia a la Clase

In [40]:
smodel = connect_to_app.SapModel;

In [41]:
full_info_software = smodel.GetProgramInfo()
print(full_info_software)

['ETABS', '20.0.0', 'Ultimate ', 0]


### 5. Se establece desbloqueo para ejecutar analisis.

In [42]:
if (smodel.GetModelIsLocked()): smodel.SetModelIsLocked(False)
else: print("Esta abierto, listo para analizar.")

Esta abierto, listo para analizar.


### 6. Ejecucion del modelo

In [9]:
smodel.Analyze.RunAnalysis()

0

### 7. Solicitud de informacion

In [30]:
response_frame_names = smodel.FrameObj.GetLabelNameList()
if response_frame_names[0]>0:
    vertices = [i for i in response_frame_names[1]] # filtro de nodos
    aristas = [j for j in response_frame_names[2]] # filtro de todos los elementos
    name_cols = list( set( [idx for idx in aristas if 'C' in idx] ) ) # filtro de columnas
    id_cols = []
    for name_col in name_cols:
        idx_col = name_col.split('C')
        # print(idx_col)
        id_cols.append( int(idx_col[1]) )
else:
    print("No hay datos en combos...")


['ETABS', '20.0.0', 'Ultimate ', 0]


### 8. Acceso a la base de datos, Filtros

In [130]:
filtro_tabla = 1 # 1:MODEL DEFINITION, 2:DESIGN DATA, 3: ....

NumberTables = 1
TableKey = []
TableName = []
ImportType = []
db_report_tables = smodel.DatabaseTables.GetAvailableTables(NumberTables, TableKey, TableName, ImportType)
# print(len(db_report_tables))
for idx in range(db_report_tables[0]):
    print(idx, '-->', db_report_tables[filtro_tabla][idx])


0 --> Analysis Options - Active Degrees of Freedom
1 --> Analysis Options - Analysis Model for Nonlinear Hinges
2 --> Analysis Options - Automatic Mesh Settings for Floors
3 --> Analysis Options - Automatic Rectangular Mesh Options for Walls
4 --> Analysis Options - Cracking Analysis Options
5 --> Analysis Options - Design and Response Recovery Options
6 --> Analysis Options - SAPFire Options
7 --> Area Assignments - Auto Edge Constraints
8 --> Area Assignments - Floor Auto Mesh Options
9 --> Area Assignments - Insertion Point
10 --> Area Assignments - Section Properties
11 --> Area Assignments - Summary
12 --> Area Load Assignments - Uniform
13 --> Area Section Property Definitions - Summary
14 --> Beam Bays
15 --> Beam Object Connectivity
16 --> Column Bays
17 --> Column Object Connectivity
18 --> Composite Beam Design Preferences - AISC 360-10
19 --> Composite Column Design Preferences - AISC 360-10
20 --> Concrete Beam Overwrites - ACI 318-08
21 --> Concrete Column Overwrites - ACI

In [128]:
# name_tbl = "Modal Participating Mass Ratios"
# data = smodel.DatabaseTables.GetTableForDisplayArray(name_tbl, GroupName="")

# cols, no_of_rows = data[2], data[3]
# data_values = np.array_split(data[4], no_of_rows)
# df = pd.DataFrame(data_values)
# df.columns = cols
# df

In [ ]:
# df = df.drop(['UZ','SumUZ'], axis=1)
# df

### 9. otros

### 10. Cerrar la coneccion y aplicacion

In [ ]:
if (connect_to_app.ApplicationExit(False)): print("No se pudo cerrar")
else: print("Se cerro con exito.")

smodel, connect_to_app = None, None
del smodel, connect_to_app
exit(-1)

## Referencias.

- [1] Interaccion con html ipywidgets: https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html
- [2] ...